In [1]:
from os import path, walk
from PyPDF2 import PdfFileReader
from PyPDF2.utils import PdfReadError
from collections import defaultdict
from re import compile, sub

In [2]:
split_to_nums = compile("([a-zA-Z]+)([0-9]+)")

In [3]:
def remove_bad(string):
    return sub('\t|\xa0', '', string.replace('\n', ' ')).strip()

In [4]:
def encounter_utility_file(filename):
    if any(name in filename for name in ['DS_Store', 'zip']):
        return True

In [5]:
TITLE = 'title'
AUTHOR = 'author'
NAME = 'name'
AFFILIATION = 'affiliation'
EMAIL = 'email'
TEXT = 'text'
REFERENCES = 'references'
REF_MARKER = '\xa0 ЛИТЕРАТУРА \xa0'
KEYWORDS_MARKER = 'Ключевые слова'
KEYWORDS = 'keywords'
ABSTRACT = 'abstract'

In [6]:
def add_paper_data(content, conference, year):
    if conference == 'Dialogue' and (year == '2000' or year == '2001'):
        parse_dialogue_2000_2001(content.split('\n'), conference, year)
    elif conference == 'Dialogue' and year == '2002':
        parse_dialogue_2002(content, conference, year)

In [7]:
def parse_dialogue_2000_2001(content, conference, year):
    parsed_data = {}
    parsed_data[TITLE] = content[0]
    parsed_data[AUTHOR] = []
    author_dict = {}
    author_dict[NAME] = content[2]
    author_dict[AFFILIATION] = content[3]
    author_dict[EMAIL] = content[4]
    parsed_data[AUTHOR].append(author_dict)
    parsed_data[TEXT] = remove_bad(' '.join(content[5:])).lower()
    parsed_data[KEYWORDS] = '-'
    parsed_data[ABSTRACT] = '-'
    data[conference][year].append(parsed_data)

In [8]:
def parse_dialogue_2002(content, conference, year):
    parsed_data = {}
    parsed_data[TITLE] = content.strip().split('\n')[0]
    parsed_data[AUTHOR] = []
    for author in content.strip().replace(parsed_data[TITLE], '').split(KEYWORDS_MARKER)[0].strip().split('\xa0'):
        split = author.strip().split('\n')
        author_dict = {}
        author_dict[NAME] = split[0]
        author_dict[AFFILIATION] = split[1]
        author_dict[EMAIL] = split[2] 
        parsed_data[AUTHOR].append(author_dict)
    parsed_data[KEYWORDS] = content.strip().replace(parsed_data[TITLE], '').split(KEYWORDS_MARKER)[1].split('\n', 1)[0].strip()
    parsed_data[ABSTRACT] = content.strip().replace(parsed_data[TITLE], '').split(KEYWORDS_MARKER)[1].split('\n', 1)[1].split('\n', 1)[0]
    parsed_data[TEXT] = remove_bad(content.strip().replace(parsed_data[TITLE], '').split(KEYWORDS_MARKER)[1].split('\n', 1)[1].split('\n', 1)[1])
    data[conference][year].append(parsed_data)

In [9]:
def search_files(root):
    for root, dirs, files in walk(root, 'r'):
        for file in files:
            with open(path.join(root, file), 'rb') as input_stream:
                if encounter_utility_file(file):
                    continue
                conference = str(root + file).split('data/')[1].split('/')[0]
                year = str(root).split('data/')[1].split('/')[1]
                try:
                    add_paper_data(input_stream.read().decode('utf-8').split('\n'), conference, year)
                except UnicodeDecodeError:
                    continue # remove
                    reader = PdfFileReader(input_stream)
                    data[conference][year].append(reader.getPage(0).extractText())

In [10]:
data = defaultdict(lambda: defaultdict(lambda: []))
contents = []

for root, dirs, files in walk(path.join('..', 'data'), 'r'):
     for file in files:
        with open(path.join(root, file), 'rb') as input_stream:
            if encounter_utility_file(file):
                continue
            conference = str(root + file).split('data/')[1].split('/')[0]
            year = str(root).split('data/')[1].split('/')[1]
            try:
                add_paper_data(input_stream.read().decode('utf-8'), conference, year)
            except UnicodeDecodeError:
                if any(name in root for name in ['all', 'online']):
                    search_files(root)
                else:   
                    continue # remove
                    reader = PdfFileReader(input_stream)
                    data[conference][year].append(reader.getPage(0).extractText())

In [11]:
from pickle import dump

with open('dialogue_2000-2001-2002.pickle', 'wb') as f:
    dump(dict(data['Dialogue']), f)